# Fred's scratchpad
December 2018

In [2]:
import tensorflow as tf
import importlib
import pge

# Reload code that may have changed since the kernel was started
importlib.reload(pge)

<module 'pge' from '/Users/freiss/research/pge/pge/__init__.py'>

In [3]:
g0_graph = tf.Graph()
with g0_graph.as_default():
    tf.constant(1, name="a0")
    tf.constant(2, name="b0")
g1_graph = tf.Graph()
with g1_graph.as_default():
    tf.constant(1, name="a1")
    tf.constant(2, name="b1")

g0 = pge.Graph(g0_graph.as_graph_def())
g1 = pge.Graph(g1_graph.as_graph_def())


In [4]:
g2_graph = tf.Graph()
with g2_graph.as_default():
    a2 = tf.placeholder(tf.float32, shape=(1,1))
    b2 = a2 + 1
    c2 = tf.identity_n([a2, a2, b2])
    d2 = tf.add(c2[0], c2[1], name="d2")
    e2 = tf.add(c2, c2, name="e2")
    
g2_graph.as_graph_def()

node {
  name: "Placeholder"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1
        }
        dim {
          size: 1
        }
      }
    }
  }
}
node {
  name: "add/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "add"
  op: "Add"
  input: "Placeholder"
  input: "add/y"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "IdentityN"
  op: "IdentityN"
  input: "Placeholder"
  input: "Placeholder"
  input: "add"
  attr {
    key: "T"
    value {
      list {
        type: DT_FLOAT
        type: DT_FLOAT
        type: DT_FLOAT
      }
    }
  }
}
node {
  name: "d2"
  op: "Add"
  input: "IdentityN"
  input: "IdentityN:1"
  attr {


In [5]:
g3_graph = tf.Graph()
with g3_graph.as_default():
    a = tf.placeholder(tf.float32, shape=(1,), name="a")
    y, idx = tf.unique(a, name="b")
    c = tf.add(y, tf.cast(idx, a.dtype), name="c")
    
    
#g3_graph.as_graph_def()

In [12]:
gd_2 = g2_graph.as_graph_def()
gd_2.node[7].input

['e2/x', 'e2/y']

In [6]:
from tensorflow.python.framework import op_def_registry
type(op_def_registry.get_registered_ops()["IdentityN"])

tensorflow.core.framework.op_def_pb2.OpDef

In [43]:
# Import the entire C API for Python use
# "Public" functions are prefixed with "TF_"
from tensorflow.python import pywrap_tensorflow as c_api
dir(c_api)

['AddControlInput',
 'AddStep',
 'AppendToFile',
 'AssertSameStructure',
 'Basename',
 'BufferedInputStream',
 'BufferedInputStream_swigregister',
 'CheckpointReader',
 'CheckpointReader_GetTensor',
 'CheckpointReader_swigregister',
 'CleanPath',
 'CopyFile',
 'CreateBufferedInputStream',
 'CreateDir',
 'CreateURI',
 'CreateWritableFile',
 'CudaSupportsHalfMatMulAndConv',
 'DeleteFile',
 'DeleteProfiler',
 'DeleteRecursively',
 'DeleteStatSummarizer',
 'Dirname',
 'DoQuantizeTrainingOnGraphDefHelper',
 'EqualAttrValueWrapper',
 'EqualGraphDefWrapper',
 'EventsWriter',
 'EventsWriter_swigregister',
 'ExtendSession',
 'Extension',
 'FileExists',
 'FileStatistics',
 'FileStatistics_swigregister',
 'Flatten',
 'FlattenForData',
 'GCluster',
 'GCluster_swigregister',
 'GItem',
 'GItem_swigregister',
 'GRAPH_DEF_VERSION',
 'GRAPH_DEF_VERSION_MIN_CONSUMER',
 'GRAPH_DEF_VERSION_MIN_PRODUCER',
 'GenerateCostReport',
 'GenerateModelReport',
 'GetChildren',
 'GetMatchingFiles',
 'GetOperationInpu

In [38]:
# Pull the Cast out of g3_graph
gd = g3_graph.as_graph_def()
cast_node_def = gd.node[2]
type(cast_node_def)

tensorflow.core.framework.node_def_pb2.NodeDef

In [42]:
dummy_graph = tf.Graph()

dummy_op = tf.Operation(cast_node_def, dummy_graph, inputs=[tf.constant(1)])

FailedPreconditionError: Input 0 ('Const_2') for 'Cast' was not previously added to ShapeRefiner.

In [56]:
first_op = g2_graph.get_operations()[0]
first_op.outputs[0]

<tf.Tensor 'Placeholder:0' shape=(1, 1) dtype=float32>

In [57]:
first_op.op_def

name: "Placeholder"
output_arg {
  name: "output"
  type_attr: "dtype"
}
attr {
  name: "dtype"
  type: "type"
}
attr {
  name: "shape"
  type: "shape"
  default_value {
    shape {
      unknown_rank: true
    }
  }
}

In [54]:
output_map = { op.name : [(t.dtype, t.shape) for t in op.outputs] 
      for op in g2_graph.get_operations() }
output_map

{'Placeholder': [(tf.float32, TensorShape([Dimension(1), Dimension(1)]))],
 'add/y': [(tf.float32, TensorShape([]))],
 'add': [(tf.float32, TensorShape([Dimension(1), Dimension(1)]))],
 'IdentityN': [(tf.float32, TensorShape([Dimension(1), Dimension(1)])),
  (tf.float32, TensorShape([Dimension(1), Dimension(1)])),
  (tf.float32, TensorShape([Dimension(1), Dimension(1)]))],
 'd2': [(tf.float32, TensorShape([Dimension(1), Dimension(1)]))],
 'e2/x': [(tf.float32,
   TensorShape([Dimension(3), Dimension(1), Dimension(1)]))],
 'e2/y': [(tf.float32,
   TensorShape([Dimension(3), Dimension(1), Dimension(1)]))],
 'e2': [(tf.float32, TensorShape([Dimension(3), Dimension(1), Dimension(1)]))]}

{'foo': 'bar'}